In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd
import numpy as np
import os

init_notebook_mode(connected=True)

In [59]:
foldername = '../Analysis/FPS/'
gt = pd.read_csv('../Groundtruth/Groundtruth.csv')
accuracy_results = pd.DataFrame({'Difficulty': ['E', 'M', 'H', 'X']})

for filename in os.listdir(foldername):
    data = pd.read_csv(foldername + filename)
    df = pd.merge(gt, data, on='File', suffixes=('_GT', '_' + filename))

    accuracy = []

    for index, row in df.iterrows():
        if row['UP_GT'] + row['DOWN_GT'] > 0:
            accuracy.append(1-(abs(row['UP_' + filename] - row['UP_GT']) + abs(row['DOWN_' + filename] - row['DOWN_GT'])) / (row['UP_GT'] + row['DOWN_GT']))
        else:
            accuracy.append(1-abs(row['UP_' + filename] - row['UP_GT']) + abs(row['DOWN_' + filename] - row['DOWN_GT']))

    df['ACCURACY'] = accuracy

    final_accuracy = df.groupby('Difficulty')['ACCURACY'].mean().reset_index()
    final_accuracy.rename(columns={'ACCURACY': 'ACCURACY_' + filename.split('.c')[0]}, inplace=True)


    accuracy_results = pd.merge(accuracy_results, final_accuracy, on='Difficulty')
    
accuracy_results = accuracy_results.T.reset_index()

new_header = accuracy_results.iloc[0] #grab the first row for the header
accuracy_results = accuracy_results[1:] #take the data less the header row
accuracy_results.columns = new_header #set the header row as the df header

accuracy_results['AVG_ACC'] = (accuracy_results['E'] + accuracy_results['M'] + accuracy_results['H']) / 3
accuracy_results['SPEED'] = accuracy_results['Difficulty'].map(lambda x: x.split('_')[-1])
accuracy_results['ALGO'] = accuracy_results['Difficulty'].map(lambda x: x.split('_')[1])

accuracy_results[['AVG_ACC','SPEED']] = accuracy_results[['AVG_ACC','SPEED']].apply(pd.to_numeric)
accuracy_results = accuracy_results.sort_values('SPEED')

print(accuracy_results)

0                 Difficulty          E         M           H          X  \
35          ACCURACY_hog_0.1  0.0333333      0.08  0.00263158          0   
52         ACCURACY_Yolo_0.1  0.0333333      0.08   0.0133459          0   
42   ACCURACY_SqueezeNet_0.1  0.0333333      0.08   0.0133459          0   
9    ACCURACY_haar_upper_0.1  0.0333333      0.08           0          0   
31    ACCURACY_MobileNet_0.1  0.0333333      0.08   0.0133459          0   
1         ACCURACY_Yolo_0.25  0.0333333      0.08    0.028244          0   
55  ACCURACY_haar_upper_0.25  0.0333333      0.08           0          0   
38         ACCURACY_hog_0.25  0.0333333      0.08  0.00333333          0   
7   ACCURACY_SqueezeNet_0.25  0.0333333      0.08   0.0371852          0   
50   ACCURACY_MobileNet_0.25  0.0333333      0.08   0.0371852          0   
46   ACCURACY_haar_upper_0.5  0.0333333      0.04      0.0125          0   
48         ACCURACY_Yolo_0.5  0.0333333      0.04   0.0527882   0.030303   
51    ACCURA

In [68]:
algos = ['MobileNet', 'SqueezeNet', 'haar', 'hog', 'Yolo']
traces = []

for algo in algos:
    trace = go.Scatter(
        y=accuracy_results[accuracy_results['ALGO'] == algo]['AVG_ACC'],
        x=accuracy_results[accuracy_results['ALGO'] == algo]['SPEED'],
        mode='lines',
        name=algo
    )
    traces.append(trace)
    
data = traces
layout = dict(
    title = 'Speed Accuracy Trade-off',
    yaxis = dict(title = 'Accuracy'),
    xaxis = dict(title = 'Speed (Frames Per Second)'),
)
fig = dict(data=data, layout=layout)
iplot(fig)